In [1]:
just_checking_integrity=False

In [2]:
import gc
import pandas as pd
import scipy.sparse as sps

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
valid=sps.load_npz("./drive/MyDrive/Recommendation_system/dataset/processed_data/URM_valid_bought.npz")
bought=valid.indices

In [5]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"] # "UCF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW", "ICBF_W"
for file in ["UCF","ICBF","Graph","ICF"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"./drive/MyDrive/Recommendation_system/dataset/candidates/traditional_recs/train/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


#del dataset_max
del df_feats
del df_max

0
loading UCF
18588456
loaded UCF
0
loading ICBF
16931962
loaded ICBF
0
loading Graph
9381571
loaded Graph
0
loading ICF
20341641
loaded ICF


In [6]:
# import ast
# submission_df=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/GRU/train/candidates.csv")
# if just_checking_integrity:
#     submission_df["score"]=submission_df["score"].apply(lambda x:x.replace("-inf","0")).apply(lambda x:x.replace("+inf","0")).apply(lambda x:x.replace("inf","0"))
#     submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# else:
#     submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# submission_df["code"]=submission_df["code"].map(ast.literal_eval)
# submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
# submission_df = submission_df.explode(['code','score', 'rank'])
# submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
# submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [7]:
# submission_df.head()

In [8]:
# import ast
# submission_df_tr=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/Transformer/train/candidates.csv")
# if just_checking_integrity:
#     submission_df_tr["score"]=submission_df_tr["score"].apply(lambda x:x.replace("-inf","0"))
#     submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# else:
#     submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
# submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
# submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
# submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
# submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [9]:
import ast
submission_df_LSTM=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/LSTM/train/candidates.csv")
if just_checking_integrity:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].apply(lambda x:x.replace("-inf","0"))
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
else:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [10]:
# dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
# dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")

dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")


In [11]:
if just_checking_integrity:
    dataset=dataset[:int(0.5*len(dataset))]

In [12]:
del dataset_max
# del submission_df
# del submission_df_tr

In [13]:
import numpy as np
sessions=np.unique(dataset["Session_Id"])
train=pd.DataFrame()
train["Session_Id"]=sessions
if just_checking_integrity:
    train["Item_ID"]=bought[:len(sessions)]
else:
    train["Item_ID"]=bought
train["target"]=1

In [14]:
gc.collect()

726

In [15]:
dataset=dataset.merge(train,on=["Session_Id","Item_ID"],how="left")
dataset.fillna(0,inplace=True)
gc.collect()

0

In [16]:
dataset["target"]=dataset["target"].astype("int8")
gc.collect()

0

In [17]:
sessions_containing_bought_among_candidates=dataset[["Session_Id","target"]].groupby("Session_Id").agg(sum).reset_index()
sessions_containing_bought_among_candidates=sessions_containing_bought_among_candidates.loc[sessions_containing_bought_among_candidates["target"]>0]

dataset=dataset[dataset['Session_Id'].isin(sessions_containing_bought_among_candidates["Session_Id"].tolist())]

In [18]:
NHITS=len(dataset['Session_Id'].unique())
HITRATE=NHITS/81600 #approximate value
HITRATE

0.7327696078431373

In [19]:
dataset.head(20)

,Session_Id,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,score_LSTM,rank_LSTM,Max_Score_UCF,Max_Score_ICBF,Max_Score_Graph,Max_Score_ICF,target
209,918383,55,0.054177,0.632362,0.005355,0.018442,0.001339,47.0,594.02826,7.204127,91.79825,1.092114,0
210,918383,122,0.025767,0.000000,0.000000,0.000000,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
211,918383,189,0.129413,0.000000,0.020602,0.278846,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
212,918383,215,0.036236,0.000000,0.000000,0.000000,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
213,918383,260,0.039900,0.000000,0.003595,0.012055,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
214,918383,320,0.040230,0.000000,0.000000,0.000000,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
215,918383,357,0.040280,0.000000,0.003286,0.012149,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0
216,918383,379,0.046153,0.637329,0.004070,0.016121,0.000850,73.0,594.02826,7.204127,91.79825,1.092114,0
217,918383,421,0.064009,0.604249,0.013832,0.319644,0.000775,80.0,594.02826,7.204127,91.79825,1.092114,0
218,918383,506,0.032975,0.493767,0.003990,0.017444,0.000000,0.0,594.02826,7.204127,91.79825,1.092114,0


In [20]:
dataset

,Session_Id,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,score_LSTM,rank_LSTM,Max_Score_UCF,Max_Score_ICBF,Max_Score_Graph,Max_Score_ICF,target
209,918383,55,0.054177,0.632362,0.005355,0.018442,0.001339,47.0,594.028260,7.204127,91.798250,1.092114,0
210,918383,122,0.025767,0.000000,0.000000,0.000000,0.000000,0.0,594.028260,7.204127,91.798250,1.092114,0
211,918383,189,0.129413,0.000000,0.020602,0.278846,0.000000,0.0,594.028260,7.204127,91.798250,1.092114,0
212,918383,215,0.036236,0.000000,0.000000,0.000000,0.000000,0.0,594.028260,7.204127,91.798250,1.092114,0
213,918383,260,0.039900,0.000000,0.003595,0.012055,0.000000,0.0,594.028260,7.204127,91.798250,1.092114,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39270860,999999,2034,0.000000,0.000000,0.000000,0.000000,0.001155,93.0,111.366875,9.421845,12.826448,0.786831,0
39270861,999999,1092,0.000000,0.000000,0.000000,0.000000,0.001129,96.0,111.366875,9.421845,12.826448,0.786831,0
39270862,999999,1513,0.000000,0.000000,0.000000,0.000000,0.001127,97.0,111.366875,9.421845,12.826448,0.786831,0
39270863,999999,827,0.000000,0.000000,0.000000,0.000000,0.001115,98.0,111.366875,9.421845,12.826448,0.786831,0


In [21]:
dataset_macro_features = pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/macro_feats_NN.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')
del dataset_macro_features

In [22]:
#dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

## K-Fold for All tradition models + LSTM models

In [23]:
from sklearn.model_selection import KFold
import lightgbm
import os
import time

start_time = time.time()

k=10


score=0
kf = KFold(n_splits=k)
index=0
for train_index, valid_index in kf.split(train):

    train_sessions=train["Session_Id"][train_index].tolist()
    valid_sessions=train["Session_Id"][valid_index].tolist()

    train_set=dataset[dataset["Session_Id"].isin(train_sessions)]
    valid_set=dataset[dataset["Session_Id"].isin(valid_sessions)]

    train_set=train_set.sort_values("Session_Id")
    valid_set=valid_set.sort_values("Session_Id")

    qids_train = train_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()
    qids_valid = valid_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()

    X_train=train_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_train=train_set["target"]

    X_valid=valid_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_valid=valid_set["target"]

    model = lightgbm.LGBMRanker(
    objective="lambdarank",
    #objective="rank_xendcg",
    #metric="mrr",
    n_estimators=2000,
    lambdarank_truncation_level=103,
    reg_alpha=1e-4,
    learning_rate=0.05,
    min_child_samples=400,
    subsample=0.8,
    subsample_freq=20
    )
    ranker=model.fit(
        X=X_train,
        y=y_train,
        group=qids_train,
        eval_set=[(X_valid, y_valid)],
        eval_group=[qids_valid],
        eval_at=(100),
        callbacks=[lightgbm.log_evaluation(),lightgbm.early_stopping(20,first_metric_only=True)],
        #verbose=10,
        eval_metric=["map","ndcg"]
    )
    ranker.booster_.save_model(f'./drive/MyDrive/Recommendation_system/rankers/model_{index}.txt')
    #Get maximum val score
    score+=np.max(ranker._evals_result["valid_0"]["map@100"])
    index+=1

end_time = time.time()

elapsed_time = end_time - start_time

[LightGBM] [Info] Total groups: 53724, total data: 25547127
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.484670 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4334
[LightGBM] [Info] Number of data points in the train set: 25547127, number of used features: 19
[LightGBM] [Info] Total groups: 6070, total data: 2865590
[1]	valid_0's map@100: 0.192274	valid_0's ndcg@100: 0.307056
Training until validation scores don't improve for 20 rounds
[2]	valid_0's map@100: 0.224199	valid_0's ndcg@100: 0.333041
[3]	valid_0's map@100: 0.231545	valid_0's ndcg@100: 0.339246
[4]	valid_0's map@100: 0.232597	valid_0's ndcg@100: 0.34011
[5]	valid_0's map@100: 0.232534	valid_0's ndcg@100: 0.340085
[6]	valid_0's map@100: 0.231463	valid_0's ndcg@100: 0.339137
[7]	valid_0's map@100: 0.23275	valid_0's ndcg@100: 0.340127
[8]	valid_0's map@100: 0.232542	valid_0'

In [24]:
final_performcance=score/k
print("overall MAP@100 = ", final_performcance)

overall MAP@100 =  0.2591062602490738


In [25]:
print("normalized score ", final_performcance*HITRATE)

normalized score  0.18986519271241567


In [26]:
print("K-Fold execution time:", elapsed_time, "seconds")

K-Fold execution time: 5486.706050634384 seconds
